In [3]:
import sklearn 

sklearn.__version__

'0.24.2'

In [4]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [35]:
sample = 1000
images = X_train[:sample].reshape(sample, 28 * 28) / 255
images.shape


(1000, 784)

In [36]:
from sklearn.preprocessing import OneHotEncoder

labels = y_train[:sample].reshape(-1, 1)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(labels)
enc.categories_


labels = enc.transform(labels).toarray()
labels[:5,:]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [40]:
test_images = X_test.reshape(len(X_test), 28 * 28) / 255
test_labels = enc.transform(y_test.reshape(-1, 1)).toarray()

In [41]:
np.random.seed(1)
relu = lambda x: (x >= 0) * x
relu2deriv = lambda x: x >= 0
alpha, iterations, hidden_size, pixels_per_image, num_labels = \
                                              (0.005, 350, 40, 784, 10)
weights_0_1 = 0.2 * np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size,num_labels)) - 0.1


In [42]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)

    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weights_1_2)
        error += np.sum((labels[i:i+1] - layer_2) ** 2)
        correct_cnt += int(np.argmax(layer_2) == \
                                        np.argmax(labels[i:i+1]))
        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)\
                                    * relu2deriv(layer_1)
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    sys.stdout.write("\r"+ \
                     "Epoch: "+str(j)+ \
                     " Error: " + str(error/float(len(images)))[0:5] +\
                     " Correct: " + str(correct_cnt/float(len(images))))

 I:349 Error:0.088 Correct:0.999

## Test dataset

In [44]:
for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    if(j % 10 == 0 or j == iterations-1):
      error, correct_cnt = (0.0, 0)

      for i in range(len(test_images)):

            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1,weights_1_2)

            error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            correct_cnt += int(np.argmax(layer_2) == \
                                            np.argmax(test_labels[i:i+1]))
      sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] +\
                 " Test-Acc:" + str(correct_cnt/float(len(test_images))))
      print()

 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Err:0.738 Test-Acc:0.686
 Test-Er

Dropout 50% of the weights

In [47]:
dropout_mask = np.random.randint(2,size=layer_1.shape)
dropout_mask

array([[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]])

You multiply layer_1 by 2. Why do you do this? Remember that layer_2 will perform a weighted sum of layer_1. Even though it’s weighted, it’s still a sum over the values of layer_1. If you turn off half the nodes in layer_1, that sum will be cut in half. 

In [48]:
import numpy, sys
np.random.seed(1)
def relu(x):
   return (x >= 0) * x

def relu2deriv(output):
   return output >= 0

alpha, iterations, hidden_size = (0.005, 300, 100)
pixels_per_image, num_labels = (784, 10)

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
   error, correct_cnt = (0.0,0)
   for i in range(len(images)):
      layer_0 = images[i:i+1]
      layer_1 = relu(np.dot(layer_0,weights_0_1))
      dropout_mask = np.random.randint(2, size=layer_1.shape)
      layer_1 *= dropout_mask * 2
      layer_2 = np.dot(layer_1,weights_1_2)

      error += np.sum((labels[i:i+1] - layer_2) ** 2)
      correct_cnt += int(np.argmax(layer_2) == \
                                      np.argmax(labels[i:i+1]))
      layer_2_delta = (labels[i:i+1] - layer_2)
      layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
      layer_1_delta *= dropout_mask

      weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
      weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

   if(j%10 == 0):
      test_error = 0.0
      test_correct_cnt = 0

      for i in range(len(test_images)):
           layer_0 = test_images[i:i+1]
           layer_1 = relu(np.dot(layer_0,weights_0_1))
           layer_2 = np.dot(layer_1, weights_1_2)

           test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
           test_correct_cnt += int(np.argmax(layer_2) == \
                                     np.argmax(test_labels[i:i+1]))

      sys.stdout.write("\n" + \
           "I:" + str(j) + \
           " Test-Err:" + str(test_error/ float(len(test_images)))[0:5] +\
           " Test-Acc:" + str(test_correct_cnt/ float(len(test_images)))+\
           " Train-Err:" + str(error/ float(len(images)))[0:5] +\
           " Train-Acc:" + str(correct_cnt/ float(len(images))))




I:0 Test-Err:0.632 Test-Acc:0.6388571428571429 Train-Err:0.896 Train-Acc:0.393
I:10 Test-Err:0.433 Test-Acc:0.786 Train-Err:0.479 Train-Acc:0.746
I:20 Test-Err:0.400 Test-Acc:0.8215 Train-Err:0.451 Train-Acc:0.783
I:30 Test-Err:0.395 Test-Acc:0.8272142857142857 Train-Err:0.424 Train-Acc:0.81
I:40 Test-Err:0.385 Test-Acc:0.8395714285714285 Train-Err:0.419 Train-Acc:0.826
I:50 Test-Err:0.394 Test-Acc:0.8359285714285715 Train-Err:0.433 Train-Acc:0.81
I:60 Test-Err:0.400 Test-Acc:0.8310714285714286 Train-Err:0.427 Train-Acc:0.803
I:70 Test-Err:0.383 Test-Acc:0.8357857142857142 Train-Err:0.410 Train-Acc:0.818
I:80 Test-Err:0.387 Test-Acc:0.8302142857142857 Train-Err:0.393 Train-Acc:0.845
I:90 Test-Err:0.378 Test-Acc:0.8209285714285715 Train-Err:0.390 Train-Acc:0.851
I:100 Test-Err:0.376 Test-Acc:0.8196428571428571 Train-Err:0.389 Train-Acc:0.838
I:110 Test-Err:0.370 Test-Acc:0.8242857142857143 Train-Err:0.379 Train-Acc:0.862
I:120 Test-Err:0.386 Test-Acc:0.8200714285714286 Train-Err:0.381 

## Mini-batch gradient descent

In [ ]:
import numpy as np
np.random.seed(1)

def relu(x):
    return (x >= 0) * x

def relu2deriv(output):
    return output >= 0

batch_size = 100
alpha, iterations = (0.001, 300)
pixels_per_image, num_labels, hidden_size = (784, 10, 100)

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))

        layer_0 = images[batch_start:batch_end]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1,weights_1_2)

        error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2)
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k+1]) == \
                    np.argmax(labels[batch_start+k:batch_start+k+1]))

            layer_2_delta = (labels[batch_start:batch_end]-layer_2) \
                                                            /batch_size
            layer_1_delta = layer_2_delta.dot(weights_1_2.T)* \
                                                     relu2deriv(layer_1)
            layer_1_delta *= dropout_mask

            weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
            weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    if(j%10 == 0):
        test_error = 0.0
        test_correct_cnt = 0

        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1, weights_1_2)



The first thing you’ll notice when running this code is that it runs much faster. This is because each np.dot function is now performing 100 vector dot products at a time. CPU architectures are much faster at performing dot products batched this way. 

There’s more going on here, however. Notice that alpha is 20 times larger than before. You can increase it for a fascinating reason. Imagine you were trying to find a city using a very wobbly compass. If you looked down, got a heading, and then ran 2 miles, you’d likely be way off course. But if you looked down, took 100 headings, and then averaged them, running 2 miles would probably take you in the general right direction.

Because the example takes an average of a noisy signal (the average weight change over 100 training examples), it can take bigger steps. You’ll generally see batching ranging from size 8 to as high as 256. Generally, researchers pick numbers randomly until they find a batch_size/alpha pair that seems to work well.
